In [1]:
from cltk import NLP
import requests
# Initialize the Latin language processor
cltk_nlp = NLP(language="lat")
cltk_nlp.analyze(text="Gallia est omnis divisa in partes tres")

e:\Miniconda\Enviroments\transformer-lang\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


‎𐤀 CLTK version '1.1.6'.
Pipeline for language 'Latin' (ISO: 'lat'): `LatinNormalizeProcess`, `LatinStanzaProcess`, `LatinEmbeddingsProcess`, `StopsProcess`, `LatinLexiconProcess`.


Doc(language='lat', words=[Word(index_char_start=None, index_char_stop=None, index_token=0, index_sentence=0, string='Gallia', pos=noun, lemma='Gallia', stem=None, scansion=None, xpos='A1|grn1|casA|gen2', upos='NOUN', dependency_relation='nsubj', governor=3, features={Case: [nominative], Gender: [feminine], Number: [singular]}, category={F: [neg], N: [pos], V: [neg]}, stop=False, named_entity=None, syllables=None, phonetic_transcription=None, definition=''), Word(index_char_start=None, index_char_stop=None, index_token=1, index_sentence=0, string='est', pos=auxiliary, lemma='sum', stem=None, scansion=None, xpos='N3|modA|tem1|gen6', upos='AUX', dependency_relation='cop', governor=3, features={Mood: [indicative], Number: [singular], Person: [third], Tense: [present], VerbForm: [finite], Voice: [active]}, category={F: [pos], N: [neg], V: [pos]}, stop=True, named_entity=None, syllables=None, phonetic_transcription=None, definition='sum\n\n (2d pers. es, or old ēs; old subj praes. siem,\n  

In [2]:
from cltk.lemmatize import LatinBackoffLemmatizer

lemmatizer = LatinBackoffLemmatizer()

def get_lemma(word):
    lemmatized_word = lemmatizer.lemmatize([word])
    return lemmatized_word[0][1]



In [3]:
latin_numerals = {
    'I': 1,
    'V': 5,
    'X': 10,
    'L': 50,
    'C': 100,
    'D': 500,
    'M': 1000
}

def translate_latin_to_normal(latin_numeral):
    total = 0
    prev_value = 0

    for numeral in reversed(latin_numeral):
        value = latin_numerals.get(numeral, 0)

        if value >= prev_value:
            total += value
        else:
            total -= value

        prev_value = value

    return total

In [4]:
class PluriumVariabiliumExceptio(Exception):
    def __init__(self, message="Plurium variabilium reperta sunt"):
        self.message = message
        super().__init__(self.message)

    def __str__(self):
        return f"Exceptio in Declaratione Variabilis: {self.message}"


class SiSententiaExceptio(Exception):
    def __init__(self, message="Si sententia nimis multos vel paucos argumentos accepit"):
        self.message = message
        super().__init__(self.message)

    def __str__(self):
        return f"Exceptio in Sententia 'Si': {self.message}"


class FranciaeExceptio(Exception):
    def __init__(self, message="In Gallia es"):
        self.message = message
        super().__init__(self.message)

    def __str__(self):
        return f"Exceptio in Francia: {self.message}"


In [5]:
stack = []
programmCounter = -1
fileContent = []

In [6]:
def findByCase(target_case, words):
    return [entry["lemma"] for entry in words if entry["pos"] == "NOUN" and target_case in str(entry['features']["Case"])]

def findNumber(words):
    return [entry["lemma"] for entry in words if entry["pos"] == "NUM" or entry["pos" ] == "ADP"]

def findObjektAttributes(stack, variableName):
    for i in stack:
        if i["object"] == variableName:
            return i
        
def is_in_france():
    response = requests.get("https://ipinfo.io")
    data = response.json()
    country = data.get("country")
    if country == "FR":
        return True
    return False



In [7]:
# Lateinische Wörter des Besitzes
lateinische_woerter_besitz = [
    "habeo",  # Ich habe
    "possideo",  # Ich besitze
    "teneo",  # Ich halte
    "obtineo",  # Ich erlange
    "adipiscor",  # Ich erlange
    "capio",  # Ich ergreife
    "conservo",  # Ich bewahre
    "retineo",  # Ich behalte
    "acquiro",  # Ich erwerbe
    "gubernio",  # Ich lenke
    # Weitere Wörter hinzufügen...
]

# Lateinische Wörter des Erhalts
getWords = [
    "accipio",  # Ich erhalte
    "recipio",  # Ich nehme an
    "obtineo",  # Ich erhalte
    "excipio",  # Ich mache eine Ausnahme für
    "capio",  # Ich fange
    "sumo",  # Ich nehme
    "recuso",  # Ich lehne ab
    "consequor",  # Ich erreiche
    "reporto",  # Ich bringe zurück
    "suscipio",  # Ich übernehme
    # Weitere Wörter hinzufügen...
]

# Lateinische Wörter des Wegnehmens
lateinische_woerter_wegnehmen = [
    "aufero",  # Ich nehme weg
    "tollo",  # Ich hebe auf
    "diripio",  # Ich plündere
    "abripio",  # Ich reiße weg
    "eripio",  # Ich entreiße
    "adimo",  # Ich nehme weg
    "privato",  # Ich entziehe
    "detraho",  # Ich entziehe
    # Weitere Wörter hinzufügen...
]

# Lateinische Wörter des Übertragens
lateinische_woerter_uebertragen = [
    "transfero",  # Ich übertrage
    "transporto",  # Ich transportiere
    "confero",  # Ich bringe zusammen
    "remitto",  # Ich sende zurück
    "committo",  # Ich begehe
    "traduco",  # Ich führe hinüber
    "permuto",  # Ich tausche
    "communico",  # Ich teile mit
    "concedo",  # Ich gewähre
    "attribuo",  # Ich schreibe zu
    # Weitere Wörter hinzufügen...
]

# Lateinische Wörter des Verlierens
loseWords = [
    "amitto",  # Ich verliere
    "perdo",  # Ich verliere
    "dilabor",  # Ich zerstreue
    "excidio",  # Ich verursache Verlust
    "deperdo",  # Ich verliere völlig
    "corrumpo",  # Ich verderbe
    "consumo",  # Ich verbrauche
    "eluo",  # Ich wasche weg
    "derelinquo",  # Ich lasse im Stich
    "decido",  # Ich falle herunter
    # Weitere Wörter hinzufügen...
]



In [25]:
def findVariableDefinitions(words) -> bool: # auf doppelte Variablen prüfen
    appearWords = [
        "venio",  # Ich komme
        "appareo",  # Ich tauche auf
        "advenio",  # Ich komme an
        "adsum",  # Ich bin anwesend
        "pareo",  # Ich erscheine
        "occuro",  # Ich begegne
        "exsisto",  # Ich entstehe
        "incipio",  # Ich beginne
        "obvenio",  # Ich begegne zufällig
        "adpropinquo",  # Ich nähere mich
        "appero",  # Ich erscheine
    ]
    if any(entry["lemma"] in appearWords for entry in words):
        target_case = '[nominative]'
        matching_words = [entry['word'] for entry in words if entry["pos"] == "NOUN" and target_case in str(entry['features']["Case"])]
        if len(matching_words) == 1:
            if any(i["object"] == matching_words[0] for i in stack):
                raise PluriumVariabiliumExceptio()
            stack.append({"object": matching_words[0], "attributes": []})
            return True
        elif len(matching_words) > 1:
            raise PluriumVariabiliumExceptio()
    return False

def findVariableAssignments(words) -> bool:
    foundNumbers = findNumber(words)
    if any(entry["lemma"]=="si" for entry in words):
        print("Found if statement")
        return False
    if len(foundNumbers) == 1:
        target_case = 'nominative'
        foundVariable = findByCase(target_case, words)
        if len(foundVariable) == 1:
            variable = foundVariable[0]
            for i in stack:
                if i["object"] == variable:
                    target_case = 'accusative'
                    foundAttributes = findByCase(target_case, words)
                    added = False
                    for attribute in i["attributes"]:
                        if attribute["name"] == get_lemma(foundAttributes[0]):
                            attribute["value"] = translate_latin_to_normal(foundNumbers[0])
                            attribute["type"] = "number"
                            added = True
                            break
                    
                    if len(foundAttributes) == 1 and not added:
                        i["attributes"].append({"name": get_lemma(foundAttributes[0]), "type": "number", "value": translate_latin_to_normal(foundNumbers[0])})
                    
                    
                    # i["value"] = translate_latin_to_normal(foundNumbers[0])
                    # i["type"] = "number"
                    break
        return True
    return False

def runProgramPart(words) -> bool:
    if addValueToVariable(words):
        return True
    elif printStatement(words):
        return True
    elif ifCondition(words):
        return True
    elif forLoop(words):
        return True
    elif findVariableDefinitions(words):
        return True
    elif findVariableAssignments(words):
        return True


    return False


def addValueToVariable(words):
    
    if any(entry["lemma"] in getWords for entry in words):
        target_case = 'nominative'
        foundVariable = findByCase(target_case, words)
        attribute = findByCase("accusative", words)
        value = findNumber(words)
        if len(foundVariable) == 1 and len(attribute) == 1 and len(value) == 1:
            for object in stack:
                if object["object"] == foundVariable[0]:
                    for attr in object["attributes"]:
                        print(attr["name"] + " " + attribute[0])
                        if attr["name"] == attribute[0]:
                            attr["value"] = int(attr["value"]) + int(translate_latin_to_normal(value[0]))
                            return True
        return True
    return False

def subtractValueFromVariable(words):
    if any(entry["lemma"] in loseWords for entry in words):
        target_case = 'nominative'
        foundVariable = findByCase(target_case, words)
        attribute = findByCase("accusative", words)
        value = findNumber(words)
        if len(foundVariable) == 1 and len(attribute) == 1 and len(value) == 1:
            for object in stack:
                if object["object"] == foundVariable[0]:
                    for attr in object["attributes"]:
                        print(attr["name"] + " " + attribute[0])
                        if attr["name"] == attribute[0]:
                            attr["value"] = int(attr["value"]) - int(translate_latin_to_normal(value[0]))
                            return True
        return True
    return False
                


def printStatement(words) -> bool:
    if any(entry["lemma"] == "dico" for entry in words):
        print("Found print statement")
        target_case = 'nominative'
        foundVariable = findByCase(target_case, words)
        attribute = findByCase("accusative", words)
        if len(foundVariable) == 1 and len(attribute) == 1:
            for object in stack:
                if object["object"] == foundVariable[0]:
                    for attr in object["attributes"]:
                        print(attr["name"] + " " + attribute[0])
                        if attr["name"] == attribute[0]:
                            add_output(attr["value"])
                            return True
        return True
    return False


def analizeIfCondition(words) -> bool:
    foundVariable = findByCase('nominative', words)
    foundAttributes = findByCase('accusative', words)
    foundNumber = findNumber(words)
    if not (len(foundVariable) == 1 and len(foundAttributes) == 1 and len(foundNumber) == 1):
        raise SiSententiaExceptio()
    
    ifCondition = False
    if any(entry["lemma"]=="habeo" for entry in words):
        for attribute in findObjektAttributes(stack, foundVariable[0])["attributes"]:
            if attribute["name"] == get_lemma(foundAttributes[0]):
                if attribute["value"] == translate_latin_to_normal(foundNumber[0]):
                    ifCondition = True
                    break
    return ifCondition



def ifCondition(words) -> bool:
    beforeComma = []
    foundComma = False
    if not any(entry["lemma"]=="si" for entry in words):
        return False
    for word in words:
        if word["word"] == ",":
            foundComma = True
        elif not foundComma:
            beforeComma.append(word)
    if len(beforeComma) == 0:
        raise SiSententiaExceptio("If statement is missing a segment")
    
    
    
    foundVariable = findByCase('nominative', beforeComma)
    foundAttributes = findByCase('accusative', beforeComma)
    foundNumber = findNumber(beforeComma)
    if not (len(foundVariable) == 1 and len(foundAttributes) == 1 and len(foundNumber) == 1):
        raise SiSententiaExceptio("If statement got to many or to few arguments")
    
    ifCondition = False
    if any(entry["lemma"]=="habeo" for entry in beforeComma):
        for attribute in findObjektAttributes(stack, foundVariable[0])["attributes"]:
            if attribute["name"] == get_lemma(foundAttributes[0]):
                if attribute["value"] == translate_latin_to_normal(foundNumber[0]):
                    ifCondition = True
                    break
    global programmCounter
    global fileContent
    programmCounter += 1
    if ifCondition:
        foundDot = False
        while not foundDot:
            analysis = cltk_nlp.analyze(text=fileContent[programmCounter])
            words = [{"word": word.string, "pos": word.upos, "features": word.features, "lemma": word.lemma} for word in analysis.words]
            runProgramPart(words)
            foundDot = any("." in word for word in fileContent[programmCounter])
            programmCounter += 1
        return True   
    else:
        print("If condition is false")
        foundDot = False
        while not foundDot:
            foundDot = any("." in word for word in fileContent[programmCounter])
            programmCounter += 1  
    return False



def forLoop(words):
    if any(entry["lemma"]=="facio" for entry in words) and any(entry["word"]=="vicibus" for entry in words):
        global programmCounter
        forLoopStart = programmCounter
        forLoopEnd = programmCounter
        analysis = cltk_nlp.analyze(text=fileContent[programmCounter])
        words = [{"word": word.string, "pos": word.upos, "features": word.features, "lemma": word.lemma} for word in analysis.words]
        number = findNumber(words)
        if len(number) == 1:
            iteration = translate_latin_to_normal(number[0])
            iteration = int(iteration)
        
            programmCounter += 1
            memory = []
            foundDot = False
            for v in range(iteration):
                while not foundDot:
                    analysis = None
                    found = False
                    for i in memory:
                        if i["line"] == programmCounter:
                            analysis = i["analysis"]
                            found = True
                    if not found:
                        analysis = cltk_nlp.analyze(text=fileContent[programmCounter])
                        memory.append({"line": programmCounter, "analysis": analysis})
                    words = [{"word": word.string, "pos": word.upos, "features": word.features, "lemma": word.lemma} for word in analysis.words]
                    runProgramPart(words)
                    foundDot = any("." in word for word in fileContent[programmCounter])
                    programmCounter += 1
                foundDot = False
                programmCounter = forLoopStart + 1

                
                
            programmCounter = forLoopEnd + 1
        return True
    

import traceback 
    
    
stack = []
programmCounter = 0
def runProgram(file="code.txt"):
    try:
        if is_in_france():
            raise FranciaeExceptio("You are in France")
        global fileContent
        with open(file) as f:
            fileContent = f.readlines()
        global programmCounter
        #for programmCounter in range(len(fileContent)):
        while programmCounter <= len(fileContent)-1:
            
            line = fileContent[programmCounter]
            print("Line: ", line)
            words = []
            analysis = cltk_nlp.analyze(text=line)
            words = [{"word": word.string, "pos": word.upos, "features": word.features, "lemma": word.lemma} for word in analysis.words]
            
            runProgramPart(words)
            programmCounter += 1
            continue
    except Exception as e:
        tracebackError = traceback.format_exc()
        traceBackArray = tracebackError.split("\n")
        for i in traceBackArray:
            add_output(i, "red")
        

    print("Program counter: ", programmCounter)
    print("stack: ", stack)    
    
    
    
    
    
    
    
import tkinter as tk
from tkinter import scrolledtext
from threading import Thread

def insert():
    text = text_field.get('1.0', 'end-1c')  # get text from the text field
    lines = text.split("\n")  # split text into lines
    with open('code.txt', 'w') as f:  # open the file
        for i, line in enumerate(lines, start=1):  # iterate over lines with line numbers
            f.write(f'{line}\n')  # write each line to the file with line number
    add_output('Text copied to code.txt with line numbers.\n', 'green')
    programThread = Thread(target=runProgram )  # create a thread for the program
    programThread.start()  # start the thread
    #runProgram('code.txt')

def add_output(text, color='black'):
    print(text)  # print the text to the console
    console.insert(tk.END, str(text)+"\n")  # insert text at the end of the console
    console.tag_add(color, 'end - %d chars' % len(str(text)), tk.END)  # apply the tag to the last inserted text
    console.tag_config(color, foreground=color)  # set the tag color

root = tk.Tk()  # create a tkinter root window
root.geometry('800x600')  # set window size

# create a scrolled text field that allows for multiple lines
text_field = scrolledtext.ScrolledText(root, wrap = tk.WORD, width = 80, height = 20)
text_field.pack(padx = 10, pady = 10)  # add the text field to the window

run_button = tk.Button(root, text = 'Run', command = insert)  # create a 'Run' button
run_button.pack(side=tk.TOP, anchor=tk.E, padx=10, pady=10)  # add the button to the window

# create a console output field below the text field
console = scrolledtext.ScrolledText(root, wrap = tk.WORD, width = 80, height = 10)
console.pack(padx = 10, pady = 10)  # add the console to the window

root.mainloop()  # start the tkinter main loop


Text copied to code.txt with line numbers.

Line:  Herkules paret.

Line:  Herkules paret.

Traceback (most recent call last):
  File "C:\Users\Benja\AppData\Local\Temp\ipykernel_25528\3134949442.py", line 261, in runProgram
    runProgramPart(words)
  File "C:\Users\Benja\AppData\Local\Temp\ipykernel_25528\3134949442.py", line 68, in runProgramPart
    elif findVariableDefinitions(words):
  File "C:\Users\Benja\AppData\Local\Temp\ipykernel_25528\3134949442.py", line 20, in findVariableDefinitions
    raise PluriumVariabiliumExceptio()
PluriumVariabiliumExceptio: Exceptio in Declaratione Variabilis: Plurium variabilium reperta sunt

Program counter:  1
stack:  [{'object': 'Herkules', 'attributes': []}]


In [9]:
stack = []
programmCounter = 0
def runProgram():
    if is_in_france():
        raise FranciaeExceptio("You are in France")


    global fileContent
    with open("text.txt") as f:
        fileContent = f.readlines()
    global programmCounter
    #for programmCounter in range(len(fileContent)):
    while programmCounter <= len(fileContent)-1:
        
        line = fileContent[programmCounter]
        print("Line: ", line)
        words = []
        analysis = cltk_nlp.analyze(text=line)
        words = [{"word": word.string, "pos": word.upos, "features": word.features, "lemma": word.lemma} for word in analysis.words]
        
        runProgramPart(words)
        programmCounter += 1
        continue
        

    print("Program counter: ", programmCounter)
    print("stack: ", stack)
    
runProgram()

Line:  Herkules paret.

Line:  Flavia apperet.

Line:  Herkules habet IX asinos.

Line:  Herkules accipit II asinos.

asinus asinus
Line:  Herkules dicit: asinus.

Found variable: ['Herkules', 'asinus']
Line:  Herkules habet II nautas.

Line:  Herkules facit III vicibus,



IndexError: list index out of range

In [ ]:
import tkinter as tk
from tkinter import scrolledtext

def run():
    text = text_field.get('1.0', 'end-1c')  # get text from the text field
    lines = text.split("\n")  # split text into lines
    with open('code.txt', 'w') as f:  # open the file
        for i, line in enumerate(lines, start=1):  # iterate over lines with line numbers
            f.write(f'{i}: {line}\n')  # write each line to the file with line number
    add_output('Text copied to code.txt with line numbers.\n', 'green')

def add_output(text, color='black'):
    console.insert(tk.END, text)  # insert text at the end of the console
    console.tag_add(color, 'end - %d chars' % len(str(text)), tk.END)  # apply the tag to the last inserted text
    console.tag_config(color, foreground=color)  # set the tag color

root = tk.Tk()  # create a tkinter root window
root.geometry('800x600')  # set window size

# create a scrolled text field that allows for multiple lines
text_field = scrolledtext.ScrolledText(root, wrap = tk.WORD, width = 80, height = 20)
text_field.pack(padx = 10, pady = 10)  # add the text field to the window

run_button = tk.Button(root, text = 'Run', command = run)  # create a 'Run' button
run_button.pack(side=tk.TOP, anchor=tk.E, padx=10, pady=10)  # add the button to the window

# create a console output field below the text field
console = scrolledtext.ScrolledText(root, wrap = tk.WORD, width = 80, height = 10)
console.pack(padx = 10, pady = 10)  # add the console to the window

root.mainloop()  # start the tkinter main loop
